In [105]:
import pandas as pd
import keras
from keras import layers
import keras_tuner
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [106]:
df_list = pd.read_csv('df_with_dummies.csv', index_col=0)

In [107]:
df_list.shape

(26915, 429)

In [108]:
df_list_Y = df_list['price']

In [109]:
df_list = df_list.drop(columns=['price'])
df_list = df_list.select_dtypes(['number'])

In [110]:
X_train, X_test, y_train, y_test = train_test_split(df_list, df_list_Y, test_size=0.3, random_state=42)

In [111]:
val_amount = int(df_list.shape[0]*0.1)
val_amount

2691

In [112]:
df_list.shape

(26915, 428)

In [113]:
val_X = X_train[0:val_amount]
val_y = y_train.iloc[0:val_amount]
X_train_part = X_train[val_amount:-1]
y_train_part = y_train.iloc[val_amount:-1]

In [114]:
callback = EarlyStopping(monitor='val_loss', patience= 3)

In [115]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2,6)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=256,
                                            step=32),
                               activation='relu'))
        if hp.Boolean('dropout_' + str(i)):
            model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=optimizers.RMSprop(learning_rate=learning_rate),
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    return model

In [116]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="WUM",
)

tuner.search_space_summary()

Search space summary
Default search space size: 6
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dropout_0 (Boolean)
{'default': False, 'conditions': []}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dropout_1 (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [117]:
tuner.search(X_train_part, y_train_part, epochs=15, validation_data=(val_X, val_y), callbacks=[callback])

Trial 10 Complete [00h 02m 08s]
val_loss: 7969.250244140625

Best val_loss So Far: 6138.557861328125
Total elapsed time: 00h 21m 57s
INFO:tensorflow:Oracle triggered exit


In [118]:
tuner.results_summary()

Results summary
Results in my_dir\WUM
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 3
units_0: 224
dropout_0: False
units_1: 64
dropout_1: False
lr: 0.0013660049365971218
units_2: 96
dropout_2: False
units_3: 192
dropout_3: True
units_4: 96
dropout_4: False
units_5: 64
dropout_5: False
Score: 6138.557861328125
Trial summary
Hyperparameters:
num_layers: 4
units_0: 128
dropout_0: False
units_1: 160
dropout_1: False
lr: 0.0009083541784678265
units_2: 192
dropout_2: False
units_3: 32
dropout_3: True
units_4: 224
dropout_4: False
units_5: 160
dropout_5: True
Score: 6254.871337890625
Trial summary
Hyperparameters:
num_layers: 3
units_0: 160
dropout_0: True
units_1: 96
dropout_1: False
lr: 0.0005894233769751197
units_2: 64
dropout_2: False
units_3: 256
dropout_3: False
Score: 6279.138916015625
Trial summary
Hyperparameters:
num_layers: 4
units_0: 192
dropout_0: False
units_1: 128
dropout_1: False
lr: 0.00029070791188026695
units_2: 32
dropout_2: False
units_3: 32
dropout_3